# SQL - Query columns in table by type

In [ ]:
# schema name
schema_name = "cidi"

# admin role
admin_role = schema_name + "_admin"
select_role = schema_name + "_select"

# ==> database table names - just like file names above, store reused database information in variables here.

# work table name
work_db_table = "dhs_lodge_history_adults"

print( "Database variables initialized at " + str( datetime.datetime.now() ) )

In [ ]:
# Database connection properties
db_host = "10.10.2.10"
db_port = -1
db_username = None
db_password = None
db_name = "appliedda"

print( "Database connection properties initialized at " + str( datetime.datetime.now() ) )

In [ ]:
# create psycopg2 connection to Postgresql

# example connect() call that uses all the possible parameters
#pgsql_connection = psycopg2.connect( host = db_host, port = db_port, database = db_name, user = db_username, password = db_password )

# for SQLAlchemy, just needed database name. Same for DBAPI?
pgsql_connection = psycopg2.connect( host = db_host, database = db_name )

print( "Postgresql connection to database \"" + db_name + "\" created at " + str( datetime.datetime.now() ) )

In [ ]:
# create a cursor that maps column names to values
pgsql_cursor = pgsql_connection.cursor( cursor_factory = psycopg2.extras.DictCursor )

print( "Postgresql cursor for database \"" + db_name + "\" created at " + str( datetime.datetime.now() ) )

In [ ]:
# make a second cursor.
work_cursor = pgsql_connection.cursor( cursor_factory = psycopg2.extras.DictCursor )
work_sql_string = ""

# SQL string to select columns in table.
sql_string = "SELECT *"
sql_string += " FROM information_schema.columns"
sql_string += " WHERE table_schema = '" + schema_name + "'"
sql_string += "     AND table_name = '" + work_db_table + "'"
sql_string += "     AND data_type LIKE '%text%'"
sql_string += ";"

print( "SQL: " + sql_string )

# run query
pgsql_cursor.execute( sql_string )

# loop over results, getting each column name, and searching within for "u'" at the start of values.
for column_info in pgsql_cursor:
    
    # get column name
    column_name = column_info[ "column_name" ]
    
    # Make SQL to look for "u'" at start of values.
    work_sql_string = "SELECT COUNT( * ) AS unicode_count"
    work_sql_string += " FROM " + schema_name + "." + work_db_table
    #work_sql_string += " WHERE " + column_name + " LIKE 'u''%'"
    work_sql_string += " WHERE " + column_name + " LIKE 'u\"%'"
    work_sql_string += ";"
    
    #print( "----> Work SQL: " + work_sql_string )
    
    # run SQL
    work_cursor.execute( work_sql_string )
    
    # unicode_count
    first_row = work_cursor.fetchone()
    unicode_count = first_row[ "unicode_count" ]
    
    print( "- " + column_name + " unicode_count = " + str( unicode_count ) )
    
#-- END loop over pgsql_cursor --#